In [1]:
!git clone https://github.com/Leejunho123/5G_project.git
%cd 5G_project/DeepLearning

Cloning into '5G_project'...
remote: Enumerating objects: 389, done.
remote: Counting objects: 100% (389/389), done.
remote: Compressing objects: 100% (243/243), done.
remote: Total 11417 (delta 294), reused 233 (delta 145), pack-reused 11028
Receiving objects: 100% (11417/11417), 245.93 MiB | 22.23 MiB/s, done.
Resolving deltas: 100% (2832/2832), done.
Checking out files: 100% (246/246), done.
/content/5G_project/DeepLearning


In [1]:
!git clone https://github.com/WegraLee/deep-learning-from-scratch-3.git
%cd deep-learning-from-scratch-3

Cloning into 'deep-learning-from-scratch-3'...
remote: Enumerating objects: 2097, done.
remote: Total 2097 (delta 0), reused 0 (delta 0), pack-reused 2097
Receiving objects: 100% (2097/2097), 32.30 MiB | 11.93 MiB/s, done.
Resolving deltas: 100% (1444/1444), done.
/content/deep-learning-from-scratch-3


## 44 매개변수를 모아두는 계층


### 44.1 parameter 클래스 구현

In [ ]:
class Parameter(Variable):
  pass


In [ ]:
import numpy as np
from dezero import Variable, Parameter

x = Variable(np.array(1.0))
p = Parameter(np.array(2.0))
y = x * p
print(isinstance(p, Parameter))
print(isinstance(x, Parameter))
print(isinstance(y, Parameter))


True
False
False


### 44.2 Layer 클래스 구현

In [ ]:
from dezero.core import Parameter

class Layer:
  def __init__(self):
    self._params = set()

  def __setattr__(self, name, value):
    if isinstance(value, Parameter):
      self._params.add(name)
    super().__setattr__(name,value)

In [ ]:
layer = Layer()
layer.p1 = Parameter(np.array(1))
layer.p2 = Parameter(np.array(2))
layer.p3 = Variable(np.array(3))
layer.p4 = 'test'

print(layer._params)

{'p2', 'p1'}


In [ ]:
for name in layer._params:
  print(name, layer.__dict__[name])

p2 variable(2)
p1 variable(1)


In [ ]:
import weakref

class Layer:
  def __init__(self):
    self._params = set()

  def __setattr__(self, name, value):
    if isinstance(value, Parameter):
      self._params.add(name)
    super().__setattr__(name,value)

  def __call__(self, *inputs):
    outputs = self.forward(*inputs)
    if not isinstance(outputs, tupe):
      outputs = (outputs,)
    self.outputs = [weakref.ref(y) for y in outputs]
    return outputs if len(outputs) > 1 else outputs[0]

  def forward(self, inputs):
    raise NotImplementedError()

  def params(self):
    for name in self._params:
      yield self.__dict__[name]

  def cleargrads(self):
    for param in self.params():
      param.cleargrad()

### 44.3 Linear 클래스 구현

In [ ]:
import numpy as np
import dezero.functions as F
from dezero.core import Parameter

class Linear(Layer):
  def __init__(self, in_size, out_size, nobias=False, dtype=np.float32):
    super().__init__()

    I, O = in_size, out_size
    W_data = np.random.randn(I, O).astype(dtype) * np.sqrt(1/ I)
    self.W = Parameter(W_data, name='W')
    if nobias:
      self.b = None
    else :
      self.b = Parameter(np.zeros(0, dtype=dtype), name = 'b')

  def forward(self, x):
    y = F.linear(x, self.W, self.b)
    return y


+ in_size를 굳이 넣지 않더라도 forward에서 자동으로 규격에 맞게 생성. 가중치는 in_size를 만들고 난 후에 설정


In [ ]:
import numpy as np
import dezero.functions as F
from dezero.core import Parameter

class Linear(Layer):
  def __init__(self, out_size, nobais = False, dtype = np.float32, in_size = None):
    super().__init()
    self.in_size = in_size
    self.out_size = out_size
    self.dtype = dtype

    self.W = Parameter(None, name='W')
    if self.in_size is not None:
      self._init_w()

    if nobias:
      self.b = None

    else :
      self.b = Parameter(np.zeros(out_size,dtype=dtype),name='b')

  def _init_W(self):
    I, O = self.in_size, self.out_size
    W_data = np.random.randn(I, O).astype(self.dtype) * np.sqrt(1 / I)
    self.W.data = W_data

  def forward(self, x):
    if self.W.data is None:
      self.in_size = x.shape[1]
      self._init_W()

    y = F.linear(x, self.W, self.b)
    return y

### 44.4 Layer를 이용한 신경망 구현

In [3]:
import numpy as np
from dezero import Variable
import dezero.functions as F
import dezero.layers as L
np.random.seed(0)
x = np.random.rand(100, 1)
y = np.sin(2 * np.pi * x) + np.random.rand(100, 1)
y = np.array(y,dtype=np.float32)
l1 = L.Linear(10)
l2 = L.Linear(1)

lr = 0.2
iters = 10000

for i in range(iters):
  z = l1(x)
  z = F.sigmoid(z)
  y_pred = l2(z)
  loss = F.mean_squared_error(y, y_pred)
  l1.cleargrads()
  l2.cleargrads()
  loss.backward()
  for l in [l1,l2]:
    for p in l.params():
      p.data = p.data - lr * p.grad.data
  if i % 1000 == 0 :
    print(loss)


variable(0.8165178515385901)
variable(0.24990280603370646)
variable(0.2460987415993917)
variable(0.2372158561118818)
variable(0.20793216013951693)
variable(0.12311919202480487)
variable(0.07888168052726025)
variable(0.07665712471092495)
variable(0.07634871510148038)
variable(0.07616853706356717)


In [4]:
import numpy as np
from dezero import Variable
import dezero.functions as F
import dezero.layers as L
np.random.seed(0)
x = np.random.rand(100, 1)
y = np.sin(2 * np.pi * x) + np.random.rand(100, 1)
y = np.array(y,dtype=np.float32)
l1 = L.Linear(10)
l2 = L.Linear(1)
z = l1(x)
z = F.sigmoid(z)
y_pred = l2(z)
loss = F.mean_squared_error(y, y_pred)
l1.cleargrads()
l2.cleargrads()
loss.backward()
for l in [l1,l2]:
  for p in l.params():
    p.data = p.data - lr * p.grad.data
print(loss)

variable(0.8165178515385901)
